### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
num_schools = school_data["school_name"].count()
student_tot = school_data_complete["student_name"].count()
budg_tot = school_data["budget"].sum()
avg_math = round(school_data_complete["math_score"].mean(),2)
avg_read = round(school_data_complete["reading_score"].mean(),2)
pass_bin = [0,70,100]
pass_labels = ["","Pass"]
school_data_complete["read_res"] = pd.cut(school_data_complete["reading_score"], pass_bin, labels=pass_labels)
school_data_complete["math_res"] = pd.cut(school_data_complete["math_score"], pass_bin, labels=pass_labels)
read_pass_count = (school_data_complete["read_res"] == "Pass").sum()
read_pass_perc = f"{round((read_pass_count / student_tot) *100, 2)}%"
math_pass_count = (school_data_complete["math_res"] == "Pass").sum()
math_pass_perc = f"{round((math_pass_count / student_tot) *100, 2)}%"
combo_pass_count = ((school_data_complete["read_res"] == "Pass") & (school_data_complete["math_res"] == "Pass")).sum()
combo_pass_perc = f"{round((combo_pass_count / student_tot) *100, 2)}%"
dist_summ_df = pd.DataFrame([[num_schools,student_tot,budg_tot,avg_math,math_pass_perc,avg_read,read_pass_perc,combo_pass_perc]], columns=["Total Schools","Total Students","Total Budget","Average Math Score","% Passing Math","Average Reading Score","% Passing Reading","% Overall Passing"])


dist_summ_df

,Total Schools,Total Students,Total Budget,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.99,72.39%,81.88,82.97%,60.8%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [8]:
#school_summ = school_data_complete[["school_name", "type", "student_name", "budget", "math_score", "reading_score"]]
group_schools_2 = school_data_complete.set_index("school_name")
group_schools = school_data_complete.groupby("school_name")
pass_fail = school_data_complete.drop(na)
group_sch_avgs = group_schools[["math_score", "reading_score"]].mean()
group_sch_stu_count = group_schools.agg(Total_Students = pd.NamedAgg(column="student_name", aggfunc="count"))
group_sch_set_vars = group_schools_2.loc[ : , ["type", "budget"]]
mer_sch_summ = pd.merge(pd.merge(group_sch_avgs,group_sch_stu_count, on="school_name"),group_sch_set_vars, on="school_name", how="left").drop_duplicates()
mer_sch_summ["Per Student Budget"] = mer_sch_summ["budget"]/mer_sch_summ["Total_Students"]

#group_schools["math_res"].value_counts(drop=False)
read_pass_count_2 = (group_schools["read_res"] == "Pass").sum()

NameError: name 'na' is not defined

In [4]:
## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type